# Assignment 2: Practical Machine Learning Project



31005 Machine Learning Spring 2019, Assignment 2



---



> Group number: Pit Wegner, Xianfeng Zhuge, Kailei Wu




## Introduction

## Exploration

## Methodology 

ddd

## Evaluation

ddd

## Conclusion

## Ethical